In [14]:
import pandas as pd
import random

In [26]:
df = pd.read_excel('test.xlsm')
df[df['Unnamed: 2']==4]  # select Unnamed: 2 value ==4 

,2020,年,Unnamed: 2,Unnamed: 3,Unnamed: 4,1900-01-01 00:00:00,1900-01-02 00:00:00,1900-01-03 00:00:00,1900-01-05 00:00:00,1900-01-06 00:00:00,...,1900-01-23 00:00:00,1900-01-24 00:00:00,1900-01-25 00:00:00,1900-01-26 00:00:00,1900-01-27 00:00:00,1900-01-28 00:00:00,1900-01-29 00:00:00,1900-01-30 00:00:00,1900-01-31 00:00:00,Unnamed: 35
4,李志謙,A,4,3,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,戴維安,B,4,3,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,吳嘉紘,C,4,3,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,郭昱,D,4,3,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
